In [ ]:
%pip install -r ../requirements.txt

In [2]:
import sys
sys.path.append('C:\\Users\\Gary\\workspace\\LangGraph_agent_test\\scr\\tools')

In [3]:
%pip install tavily-python

import os
from flask.cli import load_dotenv
from langchain.chat_models import init_chat_model

from typing import Annotated
from langchain_core.messages import HumanMessage, SystemMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from rich.console import Console
from rich.panel import Panel

from tavily_tool import tavily_tool
from count_tokens import count_tokens
from google_search import google_search

# Load environment variables from .env file
load_dotenv()  

# Load API keys from environment variable
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Initialize the OpenAI chat model
llm = init_chat_model("openai:gpt-4.1")


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Define the state for the graph, using TypedDict for type safety
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Create a new state graph for the conversation
graph_builder = StateGraph(State)

# List of tools to be used by the model
agent_tools = [tavily_tool, google_search]

# Bind the tools to the language model that the LLM can use to answer user queries
llm_with_tools = llm.bind_tools(agent_tools)

# Define the chatbot node logic
def chatbot(state: State):
    # The chatbot node invokes the model with the current messages
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Add the chatbot node to the graph
graph_builder.add_node("chatbot", chatbot)

# Create and add the tool node to the graph
tool_node = ToolNode(tools=agent_tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("count_tokens", ToolNode(tools=[count_tokens])) # Add a separate node for the count_tokens tool as it must run last


# Add conditional edges: if a tool is needed, go to the tool node
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# Start the graph at the chatbot node
graph_builder.add_edge(START, "chatbot")
# After using a tool, return to the chatbot node
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", "count_tokens")
# End the graph after the count_tokens node
graph_builder.add_edge("count_tokens", END)
# Compile the graph for execution
graph = graph_builder.compile()

In [7]:
%pip install grandalf

Note: you may need to restart the kernel to use updated packages.


In [12]:
# draw the graph
#from IPython.display import Image, display
#from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
#import pyppeteer

print(graph.get_graph().draw_ascii())
#print(graph.get_graph().draw_mermaid())
#display(Image(graph.get_graph().draw_mermaid_png()))


                    +-----------+                 
                    | __start__ |                 
                    +-----------+                 
                          *                       
                          *                       
                          *                       
                     +---------+                  
                     | chatbot |                  
                  ***+---------+....              
              ****        *         ....          
          ****            *             ...       
        **                *                ....   
+-------+         +--------------+             .. 
| tools |         | count_tokens |            ..  
+-------+         +--------------+          ..    
                               **         ..      
                                 **     ..        
                                   *   .          
                                +---------+       
                               

In [11]:

system_prompt = SystemMessage(
    content=(
        "Answer the user question and be concise. Respond with only the essential information."
        "Use both the Tavily and Google Search tools to find the answer. "
        "In your answer, cite both results separately and show their source. "
        "Also include a details of token usage and the total number of characters used at the end of your answer" 
        " Your reply should follow the format below:"
        " <Answer to the user question> "
        " "
        " <Answer from Tavily> : source <URL> "
        " <Answer from Google Search> : source <URL> "
        " "
        " Tool (Tokens used: <estimated_token_count>, Characters used: <character_count>). These values must be obtained from the count_tokens tool without alteration."
        " LLM (Tokens used: <total_tokens>). This information must be obtained from the LLM response looking for the response metadata for the total_tokens values."
    )
)

question = HumanMessage(content="What was the largest stock market crash and why?")

# Prepend the system prompt to the messages
messages = [system_prompt, question]

# Invoke the graph with the initial messages
answer = graph.invoke({"messages": messages})


print(answer["messages"][-1].content)

The largest stock market crash is widely considered to be the Wall Street Crash of 1929, also known as the Great Crash. It began in late October 1929, with the most significant declines occurring on October 24th (Black Thursday), October 28th (Black Monday), and October 29th (Black Tuesday). The crash was triggered by a combination of speculative bubble, excessive use of margin (borrowing to buy stocks), and panic selling. It marked the start of the Great Depression.

Answer from Tavily: The largest stock market crash is typically identified as the Wall Street Crash of 1929, which led to the Great Depression. The crash was driven by speculation, over-leveraged investments, low confidence, and panic selling. : source https://www.investopedia.com/articles/07/marketcrash.asp

Answer from Google Search: The Wall Street Crash of 1929 is considered the biggest stock market crash in history, largely caused by speculation, credit expansion, and loss of confidence. : source https://www.history.